In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os

# zip_files = []
# file_names = []
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
#         zip_files.append(os.path.join(dirname, filename))
#         file_names.append(filename.replace('.zip', ''))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# !mkdir /dataset
# from zipfile import ZipFile
# import pandas as pd

# for file, name in zip(zip_files, file_names):
#     if '.zip' in file:
#         zip_file = ZipFile(file)
#         df = pd.read_csv(zip_file.open(f'{name}'))
#         df.to_csv(f'/dataset/{name}', index=False)

# !ls /dataset

# Read dataset

In [3]:
# order_products_prior = pd.read_csv('/dataset/order_products__prior.csv')
# aisles = pd.read_csv('/dataset/aisles.csv')
# products = pd.read_csv('/dataset/products.csv')
# departments = pd.read_csv('/dataset/departments.csv')

In [4]:
# order_products_prior

In [5]:
# order_id = order_products_prior['order_id'].unique()
# len(order_id)


# import numpy as np
# order_id = np.random.choice(order_id,50000)
# order_products_prior = order_products_prior[np.isin(order_products_prior.order_id, order_id)]
# order_products_prior

In [6]:
# num_product = products.product_id.nunique()
# print(num_product)
# products

In [7]:
# num_aisle = aisles.aisle_id.nunique()
# print(num_aisle)
# aisles

In [8]:
# num_depart = departments.department_id.nunique()
# print(num_depart)
# departments

In [9]:
# porder_products_prior = order_products_prior.merge(products, on='product_id')
# orders = porder_products_prior.groupby('order_id')
# orders.ngroups

In [10]:
# from tqdm.notebook import tqdm
# orders = porder_products_prior.groupby('order_id')
# process = tqdm(desc='Filtering data....', total = len(orders))
# def filter_group(group):
#     # lọc đi những:
#         # tất cả sp đến từ 1 aisle_id hoặc department_id:
#         # có số lượng department_id = 1
#         # có số lượng aisle_id = 1
#         # có số lượng product <5
#     process.update(1)
#     return group.department_id.nunique()>1 and group.aisle_id.nunique()>1  and group.product_id.nunique()>5

# orders = orders.filter(filter_group)
# orders = orders[['order_id', 'product_id', 'aisle_id', 'department_id']]
# orders

In [11]:
# orders.to_csv('/dataset/samples_filtered_44k_orders.csv', index=False)
# orders = orders.groupby('order_id')

In [12]:
# from copy import copy
# import numpy as np
# from tqdm.notebook import tqdm

# process = tqdm(desc='Creating X-Y data....', total=len(orders))
# num_sample = tqdm(desc='Number of sample ')


# def to_order(group):
#     rs = []
#     for aisle_id in group.aisle_id.unique().tolist():
#         Ys = group[group.aisle_id==aisle_id]
#         Xs = group[group.aisle_id!=aisle_id]
#         x_product_ids = Xs.product_id.tolist()
#         x_aisle_ids = Xs.aisle_id.tolist()
#         x_department_ids = Xs.department_id.tolist()
#         rs.append({
#             'x-product_ids':x_product_ids,
#             'x-aisle_ids':x_aisle_ids,
#             'x-department_ids': x_department_ids,
#             'y-aisle_id':aisle_id,
#         })
#         num_sample.update(1)
#     process.update(1)
#     return rs

# # apply the function to each group and combine the results
# results = orders.apply(to_order).explode().tolist()
# len(results)

In [13]:
# import pickle

# with open('train.pck', 'wb+') as f:
#     pickle.dump((num_product, num_aisle, num_depart, results), f)

# (num_product, num_aisle, num_depart)

In [14]:
# import json 
# with open('sample-train-400k.json', 'w+') as f:
#     json.dump(results, f, indent=2)

# Create training data

In [15]:
import pickle
import os

if os.path.isfile('/kaggle/input/train-600k/train.pck'):
    with open('/kaggle/input/train-600k/train.pck', 'rb') as f:
        (num_product, num_aisle, num_depart, results) = pickle.load(f)
else:
    with open('train.pck', 'rb') as f:
        (num_product, num_aisle, num_depart, results) = pickle.load(f)

(num_product, num_aisle, num_depart)

(49688, 134, 21)

In [16]:
import random

random.shuffle(results)

results = results[:35000]

len(results)

35000

In [17]:
from tqdm.notebook import tqdm
import numpy as np
from scipy.sparse import csr_matrix


X = []
X_aisle_only = []
y_aisle_label = []
for idx, basket in tqdm(enumerate(results)):
    x_product_id = np.array([0,]* (num_product))
    x_aisle_id =  np.array([0,] * (num_aisle))
    unique, counts = np.unique(basket['x-product_ids'], return_counts=True)
    x_product_id[unique-1] = counts
    
    unique, counts = np.unique(basket['x-aisle_ids'], return_counts=True)    
    x_aisle_id[unique-1] = counts
    X_aisle_only.append(x_aisle_id)
    x = np.concatenate((x_product_id, x_aisle_id), axis=0)
    X.append(x)
    y_aisle_label.append(basket['y-aisle_id']) 

X = np.stack(X, axis=0)
X = csr_matrix(X)
X_aisle_only = np.stack(X_aisle_only, axis=0)
y = np.array(y_aisle_label) -1 # convert index to starting from 0
X.shape, y.shape

0it [00:00, ?it/s]

((35000, 49822), (35000,))

# Comprarative models
___
## Preprocessing techs:
<!-- * Do nothing (dense vector) -->
<!-- * Chi2 feature engineering -->
* Aisle_id only
* Product_id + Aisle_id -> TruncatedSVD 1000 (Halko, et al. (2009). “Finding structure with randomness: Stochastic algorithms for constructing approximate matrix decompositions”)
<!-- * PCA (Implements the probabilistic PCA model from: Tipping, M. E., and Bishop, C. M. (1999). “Probabilistic principal component analysis”. Journal of the Royal Statistical Society: Series B (Statistical Methodology), 61(3), 611-622. via the score and score_samples methods.) -->
<!-- * FastICA (A. Hyvarinen and E. Oja, Independent Component Analysis: Algorithms and Applications, Neural Networks, 13(4-5), 2000, pp. 411-430.) -->

In [18]:
import numpy as np
from sklearn.model_selection import train_test_split

idxs = np.arange(X.shape[0])
np.random.shuffle(idxs)
thresh = int((1-0.33)*len(idxs))
train_idx = idxs[:thresh]
test_idx = idxs[thresh:]

X_train, X_test, y_train, y_test = X[train_idx, :], X[test_idx, :], y[train_idx], y[test_idx]
# DO nothing
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print('Train test split')

# Chi2
# top_feat = 1000
# from sklearn.feature_selection import chi2
# bin_X_train = np.where(X_train.toarray()>0, 1, 0)
# print('create bin')
# chi2_val, p_value = chi2(bin_X_train, y_train)
# top_feat_idx = np.argsort(chi2_val)[-top_feat:]

# chi2X_train = X_train[:, top_feat_idx]
# chi2X_test = X_test[:, top_feat_idx]
# print('Chi2')

# TruncatedSVD
from sklearn.decomposition import TruncatedSVD
top_feat = 500
svd = TruncatedSVD(n_components=top_feat, n_iter=1, random_state=42)
svd.fit(X_train)
svdX_train = svd.transform(X_train)
svdX_test = svd.transform(X_test)
print('TruncatedSVD')

# MiniBatchSparsePCA
# from sklearn.decomposition import PCA

# top_feat = 1000
# pca = PCA(n_components=top_feat, random_state=42)
# pca.fit(X_train.toarray())
# pcaX_train = pca.transform(X_train.toarray())
# pcaX_test = pca.transform(X_test.toarray())
# print('PCA')

# FastICA
# from sklearn.decomposition import FastICA

# top_feat = 1000
# pca = FastICA(n_components=top_feat, random_state=42, max_iter=10)
# pcaX_train = pca.fit_transform(X_train.toarray())
# pcaX_test = pca.transform(X_test.toarray())
# print('FastICA')

X_aisle_only_train, X_aisle_only_test = X_aisle_only[train_idx, :], X_aisle_only[test_idx, :]

Train test split
TruncatedSVD


In [19]:
X.shape, X_aisle_only.shape, svdX_train.shape, X_aisle_only_train.shape


((35000, 49822), (35000, 134), (23449, 500), (23449, 134))

In [20]:
preprocess = [
#     ('do-nothing', X_train, X_test, y_train, y_test),
#     ('chi2-top1000', chi2X_train, chi2X_test, y_train, y_test),
    ('svd-500', svdX_train, svdX_test, y_train, y_test),
    ('aisle_only', X_aisle_only_train, X_aisle_only_test, y_train, y_test)
#     ('FastICA-1000', pcaX_train, pcaX_test, y_train, y_test),
#     ('pca-1000', pcaX_train, pcaX_test, y_train, y_test),
]

## CLF models
* MLP
* RF
* DTree
* LR

In [21]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

clfs = [
    ('lightgbm', LGBMClassifier()),
        ('dtree', DecisionTreeClassifier()),
        ('rf', RandomForestClassifier(10)),
        ('mlp', MLPClassifier()),
        ('logreg', LogisticRegression())
]

In [22]:
import numpy as np

def top_k_accuracy(y_pred, y_true, k):
    # Get the top k predicted labels for each example
    top_10_labels = np.argsort(y_pred, axis=1)[:, -k:]

    # Check if the true label is in the top k predicted labels for each example
    is_in_top_k = np.any(top_10_labels == y_true.reshape(-1, 1), axis=1)

    # Calculate the top k accuracy
    top_k_acc = np.mean(is_in_top_k)

    return top_k_acc


In [23]:
from tqdm.notebook import tqdm
import pandas as pd

process = tqdm(desc='Testing....', total=len(preprocess)*len(clfs))

bench_results = []
for (preprocess_name, x_train, x_test, y_train, y_test) in preprocess:
    for idx, (model_name, clf) in enumerate(clfs):
        clf.fit(x_train.astype(np.float32), y_train)
        probs = clf.predict_proba(x_test.astype(np.float32))
        acc = {f'acc@{k}': top_k_accuracy(probs, np.array(y_test), k) for k in [5,10,15,20]}
        rs = {
            'preprocessing': preprocess_name,
            'clf': model_name,
            **acc
        }
        bench_results.append(rs)
        process.update(1)
        
bench_results = pd.DataFrame(bench_results)
bench_results

Testing....:   0%|          | 0/10 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/co

,preprocessing,clf,acc@5,acc@10,acc@15,acc@20
0,svd-500,lightgbm,0.078608,0.095230,0.124578,0.161285
1,svd-500,dtree,0.060687,0.076357,0.106657,0.134707
2,svd-500,rf,0.149771,0.227686,0.244741,0.270799
3,svd-500,mlp,0.232015,0.334343,0.416414,0.484374
4,svd-500,logreg,0.315211,0.431911,0.521167,0.589906
5,aisle_only,lightgbm,0.063631,0.079907,0.111159,0.131417
6,aisle_only,dtree,0.085447,0.101290,0.131331,0.158688
7,aisle_only,rf,0.203532,0.273569,0.291057,0.317895
8,aisle_only,mlp,0.287940,0.398580,0.481170,0.555450
9,aisle_only,logreg,0.342395,0.461172,0.545061,0.614233


In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


class DeepModelWrapper:
    
    def predict_proba(self, test_x, batch_size=256):
        pred = self.predict(
            test_x,
            batch_size=batch_size,
        )
        return pred

class NeuralDecisionTree(keras.Model, DeepModelWrapper):
    def __init__(self, depth, num_features, used_features_rate, num_classes):
        super().__init__()
        self.depth = depth
        self.num_leaves = 2 ** depth
        self.num_classes = num_classes

        # Create a mask for the randomly selected features.
        num_used_features = int(num_features * used_features_rate)
        one_hot = np.eye(num_features)
        sampled_feature_indicies = np.random.choice(
            np.arange(num_features), num_used_features, replace=False
        )
        self.used_features_mask = one_hot[sampled_feature_indicies]

        # Initialize the weights of the classes in leaves.
        self.pi = tf.Variable(
            initial_value=tf.random_normal_initializer()(
                shape=[self.num_leaves, self.num_classes]
            ),
            dtype="float32",
            trainable=True,
        )

        # Initialize the stochastic routing layer.
        self.decision_fn = layers.Dense(
            units=self.num_leaves, activation="sigmoid", name="decision"
        )

    def call(self, features):
        batch_size = tf.shape(features)[0]

        # Apply the feature mask to the input features.
        features = tf.matmul(
            features, self.used_features_mask, transpose_b=True
        )  # [batch_size, num_used_features]
        # Compute the routing probabilities.
        decisions = tf.expand_dims(
            self.decision_fn(features), axis=2
        )  # [batch_size, num_leaves, 1]
        # Concatenate the routing probabilities with their complements.
        decisions = layers.concatenate(
            [decisions, 1 - decisions], axis=2
        )  # [batch_size, num_leaves, 2]

        mu = tf.ones([batch_size, 1, 1])

        begin_idx = 1
        end_idx = 2
        # Traverse the tree in breadth-first order.
        for level in range(self.depth):
            mu = tf.reshape(mu, [batch_size, -1, 1])  # [batch_size, 2 ** level, 1]
            mu = tf.tile(mu, (1, 1, 2))  # [batch_size, 2 ** level, 2]
            level_decisions = decisions[
                :, begin_idx:end_idx, :
            ]  # [batch_size, 2 ** level, 2]
            mu = mu * level_decisions  # [batch_size, 2**level, 2]
            begin_idx = end_idx
            end_idx = begin_idx + 2 ** (level + 1)

        mu = tf.reshape(mu, [batch_size, self.num_leaves])  # [batch_size, num_leaves]
        probabilities = keras.activations.softmax(self.pi)  # [num_leaves, num_classes]
        outputs = tf.matmul(mu, probabilities)  # [batch_size, num_classes]
        return outputs
    
class NeuralDecisionForest(keras.Model, DeepModelWrapper):
    def __init__(self, num_trees, depth, num_features, used_features_rate, num_classes):
        super().__init__()
        self.ensemble = []
        # Initialize the ensemble by adding NeuralDecisionTree instances.
        # Each tree will have its own randomly selected input features to use.
        for _ in range(num_trees):
            self.ensemble.append(
                NeuralDecisionTree(depth, num_features, used_features_rate, num_classes)
            )
        self.num_classes = num_classes

    def call(self, inputs):
        # Initialize the outputs: a [batch_size, num_classes] matrix of zeros.
        batch_size = tf.shape(inputs)[0]
        outputs = tf.zeros([batch_size, self.num_classes])

        # Aggregate the outputs of trees in the ensemble.
        for tree in self.ensemble:
            outputs += tree(inputs)
        # Divide the outputs by the ensemble size to get the average.
        outputs /= len(self.ensemble)
        return outputs

In [25]:
import pandas as pd

bench_results = []
process = tqdm(desc='Testing....', total=len(preprocess))
for (preprocess_name, x_train, x_test, y_train, y_test) in preprocess:
    clf = NeuralDecisionForest(num_trees=20, 
                     depth=5, 
                     num_features = x_train.shape[-1], 
                     used_features_rate= 0.5, 
                     num_classes=num_aisle)
    clf.build(input_shape=x_train.shape)
    clf.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.01),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    )

    clf.fit(x_train, y_train, epochs=10, batch_size=164)
    probs = clf.predict_proba(x_test.astype(np.float32))
    acc = {f'acc@{k}': top_k_accuracy(probs, np.array(y_test), k) for k in [5,10,15,20]}
    rs = {
        'preprocessing': preprocess_name,
        'clf': 'NeuralDecisionForest',
        **acc
    }
    bench_results.append(rs)
    process.update(1)


Testing....:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1/10
143/143 [==============================] - 26s 53ms/step - loss: 4.5092
Epoch 2/10
143/143 [==============================] - 7s 52ms/step - loss: 4.1782
Epoch 3/10
143/143 [==============================] - 8s 53ms/step - loss: 4.0833
Epoch 4/10
143/143 [==============================] - 8s 54ms/step - loss: 4.0132
Epoch 5/10
143/143 [==============================] - 8s 54ms/step - loss: 3.9537
Epoch 6/10
143/143 [==============================] - 8s 54ms/step - loss: 3.8960
Epoch 7/10
143/143 [==============================] - 8s 56ms/step - loss: 3.8370
Epoch 8/10
143/143 [==============================] - 8s 55ms/step - loss: 3.7757
Epoch 9/10
143/143 [==============================] - 8s 55ms/step - loss: 3.7132
Epoch 10/10
46/46 [==============================] - 4s 40ms/step
Epoch 1/10
143/143 [==============================] - 26s 45ms/step - loss: 4.5004
Epoch 2/10
143/143 [==============================] - 6s 45ms/step - loss: 4.2252
Epoch 3/10
143/143 [==========

In [26]:
bench_results = pd.DataFrame(bench_results)
bench_results

,preprocessing,clf,acc@5,acc@10,acc@15,acc@20
0,svd-500,NeuralDecisionForest,0.301619,0.410181,0.492511,0.560904
1,aisle_only,NeuralDecisionForest,0.320145,0.426457,0.506190,0.572158
